<a href="https://colab.research.google.com/github/Npkh/Marketing-Setup-New-Campaigns-Bulk-File/blob/main/T%E1%BA%A0O_CAMPAIGN_CHO_S%E1%BA%A2N_PH%E1%BA%A8M_M%E1%BB%9AI_CANAMAX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **HƯỚNG DẪN CÁC BƯỚC THỰC HIỆN** #
1. Tạo 1 file excel có 04 sheet: "lv1", "lv2", "negative", "competitor_asin" - CHÚ Ý: TÊN SHEET VIẾT THƯỜNG HẾT. [SAMPLE](https://docs.google.com/spreadsheets/d/1Hf_tC6ROLWHD-u4X0fp1Ke3_UrPW-2_s5gjxGaQG10I/edit#gid=1601468489).
2. LƯU Ý: Nếu sản phẩm không có keyword lv1 hoặc không có negative phrase, thì xóa hết dữ liệu ở sheet "lv1" và sheet "negative" chứ không xóa sheet.
3. SAVE VÀ UPLOAD LÊN ĐÂY
4. Chú ý đoạn nhập Portfolio ID vì nó là ID chứ không phải tên Porfolio. Tra cứu Portfolio ID ở đây --> [PORTFOLIO ID](https://docs.google.com/spreadsheets/d/1p6q4hy6duSR8RYQZjLTPbMVDtHjxMdz3JkDu_RCIJbw/edit#gid=0)
5. Sử dụng file tải về upload lên [Bulk Amazon](https://advertising.amazon.com/bulksheet/HomePage)

# **REQUIRED PACKAGE** #

In [ ]:
import pandas as pd
import re
from google.colab import files

# **NHẬP THÔNG TIN SẢN PHẨM** #

In [ ]:
market = input("Nhập thị trường (US hoặc CA): ")
product_name = input("Nhập tên sản phẩm: ")
SKU = input("Nhập SKU của sản phẩm: ")
asin = input("Nhập ASIN của sản phẩm: ")
portfolio = input("Nhập Portfolio ID quảng cáo của sản phẩm: ")
start_date = input("Nhập ngày bắt đầu campaign (Format 'yyyymmdd'): ")
target_asins = input("Nhập danh sách ASINs collection (Format: ASIN1, ASIN2, ASIN3,...)")
image = input("Nhập link ảnh: ")
video = input("Nhập link video: ")

Nhập thị trường (US hoặc CA): US
Nhập tên sản phẩm: 60+10 uf 370-440VAC
Nhập SKU của sản phẩm: CA-LT12-CP08
Nhập ASIN của sản phẩm: B0CQP434XC
Nhập Portfolio ID quảng cáo của sản phẩm: 237648656264189
Nhập ngày bắt đầu campaign (Format 'yyyymmdd'): 20240503
Nhập danh sách ASINs collection (Format: ASIN1, ASIN2, ASIN3,...)
Nhập link ảnh: 
Nhập link video: https://advertising.amazon.com/creative-assets/preview/amzn1.assetlibrary.asset1.f1d7a34584feff3f64c4bd4576d48739/version_v1?entityId=ENTITY2NZH287KPH8V8


# **UPLOAD VÀ XỬ LÝ FILE KEYWORD** #

In [ ]:
uploaded = files.upload()
dfs = {}

for file_name in uploaded.keys():
    xls = pd.ExcelFile(file_name)
    sheet_names = xls.sheet_names

    for sheet_name in sheet_names:
        dfs[sheet_name] = pd.read_excel(file_name, sheet_name=sheet_name)

lv1_keyword = dfs['lv1']
lv2_keyword = dfs['lv2']
negative_keyword = dfs['negative']
competitor_asin = dfs['competitor_asin']

Saving 60+10 uf 370-440VAC.xlsx to 60+10 uf 370-440VAC (1).xlsx


In [ ]:
#Xây dựng hàm xử lý ký tự đặc biệt
def process_text(text):
    if isinstance(text, str):
        cleaned_text = re.sub(r'[()#@^",/%*$]', '', text)

        cleaned_text = ' '.join(cleaned_text.split())

        return cleaned_text
    return text

In [ ]:
if not competitor_asin.empty:
  competitor_asin = competitor_asin.drop_duplicates()
  competitor_asin['SP_ASIN'] = competitor_asin['ASIN'].apply(lambda x: f'asin="{x}"')
  competitor_asin['SD_ASIN'] = competitor_asin['ASIN'].apply(lambda x: f'asin="{x}"')

In [ ]:
if market == "US":
  brand_entity = 'ENTITY26ESCR7Q7N1T7'
  brand_logo = 'amzn1.assetlibrary.asset1.8845b443b784a439176b85cfa210091c:version_v1'
elif market == "CA":
  brand_entity = 'ENTITY11SCWDPTJLJZ'
  brand_logo = 'amzn1.assetlibrary.asset1.7afa6e298852e820a6c37fbd22ce3724:version_v1'

# **TẠO DATA NEGATIVE KEYWORDS CHO CÁC CAMPAIGN** #

In [ ]:
start_date_float = float(start_date)

if not lv1_keyword.empty:
  lv1_keyword["Keyword"] = lv1_keyword["Keyword"].astype(str)
  lv1_keyword['Keyword'] = lv1_keyword['Keyword'].apply(process_text)
  lv1_keyword["Group"] = lv1_keyword["Group"].str.lower()
  lv1_keyword = lv1_keyword.drop_duplicates()


if not lv2_keyword.empty:
  lv2_keyword["Keyword"] = lv2_keyword["Keyword"].astype(str)
  lv2_keyword['Keyword'] = lv2_keyword['Keyword'].apply(process_text)
  lv2_keyword["Group"] = lv2_keyword["Group"].str.lower()
  lv2_keyword = lv2_keyword.drop_duplicates()

if not negative_keyword.empty:
    negative_keyword["Keyword"] = negative_keyword["Keyword"].astype(str)
    negative_keyword['Keyword'] = negative_keyword['Keyword'].apply(process_text)
    negative_keyword = negative_keyword.drop_duplicates()
    negative_keyword['MatchType'] = 'negative phrase'

# **TẠO CẤU TRÚC DỮ LIỆU CHO TỪNG LOẠI CAMPAIGN** #

## *Cấu Trúc Dữ Liệu Của Các Camp SP Và SD* ##

In [ ]:
data = {
    'Product': [],
    'Entity': [],
    'Operation': [],
    'Campaign Id': [],
    'Ad Group Id': [],
    'Portfolio Id': [],
    'Ad Id': [],
    'Keyword Id': [],
    'Product Targeting Id': [],
    'Campaign Name': [],
    'Ad Group Name': [],
    'Start Date': [],
    'End Date': [],
    'Targeting Type': [],
    'State': [],
    'Daily Budget': [],
    'sku': [],
    'asin': [],
    'Ad Group Default Bid': [],
    'Bid': [],
    'Keyword Text': [],
    'Match Type': [],
    'Bidding Strategy': [],
    'Placement': [],
    'Percentage': [],
    'Product Targeting Expression': []
}

## *Cấu Trúc Dữ Liệu Của Các Camp SB* ##

In [ ]:
data_sb = {
    'Product': [],
    'Entity': [],
    'Operation': [],
    'Campaign ID': [],
    'Portfolio ID': [],
    'Ad Group ID': [],
    'Ad ID': [],
    'Keyword ID': [],
    'Product Targeting ID': [],
    'Campaign name': [],
    'Ad group name': [],
    'Ad name': [],
    'Start date': [],
    'End date': [],
    'State': [],
    'Brand Entity ID': [],
    'Budget type': [],
    'Budget': [],
    'Bid optimization': [],
    'Product location': [],
    'Bid': [],
    'Placement': [],
    'Percentage': [],
    'Keyword text': [],
    'Match type': [],
    'Native language keyword': [],
    'Native language locale': [],
    'Product targeting expression': [],
    'Landing page URL': [],
    'Landing page ASINs': [],
    'Landing page type': [],
    'Brand name': [],
    'Consent to translate': [],
    'Brand logo asset ID': [],
    'Brand logo crop': [],
    'Custom images': [],
    'Creative headline': [],
    'Creative ASINs': [],
    'Video asset IDs': [],
    'Sub-pages': []
}

# **SPONSORED PRODUCTS** #

## *Auto Campaign* ##

In [ ]:
df_auto = pd.DataFrame(data)
auto_campaign_name = product_name + " " + SKU + " - Auto"

# Entity = "Campaign"
auto_campaign_row = {
    'Entity': "Campaign",
    'Campaign Id': auto_campaign_name,
    'Portfolio Id': portfolio,
    'Campaign Name': auto_campaign_name,
    'Start Date': start_date_float,
    'Targeting Type': "Auto",
    'Daily Budget': 10,
    'Bidding Strategy': "Dynamic bids - down only"}

# Entity = "Ad group"
auto_ad_group_row = {
    'Entity': "Ad group",
    'Campaign Id': auto_campaign_name,
    'Ad Group Id': auto_campaign_name,
    'Ad Group Name': auto_campaign_name,
    'Ad Group Default Bid': 0.5}

# Entity = "Product ad"
auto_product_ad_row = {
    'Entity': "Product ad",
    'Campaign Id': auto_campaign_name,
    'Ad Group Id': auto_campaign_name,
    'sku': SKU}
df_auto = pd.concat([df_auto, pd.DataFrame([auto_campaign_row, auto_ad_group_row, auto_product_ad_row])], ignore_index=True)

## *Research Campaign* ##

### Xây dựng bộ keyword chạy quảng cáo của Research Campaign ###

In [ ]:
research_keyword = pd.concat([lv1_keyword, lv2_keyword], ignore_index=True)
research_keyword['Group'] = research_keyword['Group'].str.lower()
research_keyword['MatchType'] = research_keyword['Group'].astype(str).apply(lambda x: 'negative exact' if 'spf' in x else 'broad')
research_keyword['bid'] = research_keyword['MatchType'].apply(lambda x: 0.1 if 'broad' in x else '')

In [ ]:
research_keyword_condition = research_keyword['Keyword'].astype(str).apply(lambda x: len(x.split())) > 10
research_keyword = research_keyword[~research_keyword_condition]

### Xây dựng data quảng cáo của Research Campaign ###

In [ ]:
# Tạo DataFrame từ dict
df_research = pd.DataFrame(data)

#Entity = "Campaign"
research_campaign_name = product_name + " " + SKU + " - Research"
research_campaign_row = {
    'Entity': "Campaign",
    'Campaign Id': research_campaign_name,
    'Portfolio Id': portfolio,
    'Campaign Name': research_campaign_name,
    'Start Date': start_date_float,
    'Targeting Type': "Manual",
    'Daily Budget': 10,
    'Bidding Strategy': "Dynamic bids - down only"}

#Entity = "Ad group"
research_ad_group_row = {
    'Entity': "Ad group",
    'Campaign Id': research_campaign_name,
    'Ad Group Id': research_campaign_name,
    'Ad Group Name': research_campaign_name,
    'Ad Group Default Bid': 0.75}

#Entity = "Product ad"
research_product_ad_row = {
    'Entity': "Product ad",
    'Campaign Id': research_campaign_name,
    'Ad Group Id': research_campaign_name,
    'sku': SKU}

df_research = pd.concat([df_research, pd.DataFrame([research_campaign_row, research_ad_group_row, research_product_ad_row])], ignore_index=True)

# Entity = "Negative Keyword"
for index, row in negative_keyword.iterrows():
    negative_keyword_text = row['Keyword']
    negative_match_type = row['MatchType']
    negative_row = {
        'Entity': 'Negative keyword',
        'Campaign Id': research_campaign_name,
        'Keyword Text': negative_keyword_text,
        'Match Type': negative_match_type}
    df_research = pd.concat([df_research, pd.DataFrame([negative_row])], ignore_index=True)

# Entity = "Keyword" - Các keyword được chạy quảng cáo và negative trong ad group
for index, row in research_keyword.iterrows():
    research_keyword_text = row['Keyword']
    research_match_type = row['MatchType']
    bid = row['bid']
    new_research_keyword_row = {
        'Entity': 'Keyword',
        'Campaign Id': research_campaign_name,
        'Ad Group Id': research_campaign_name,
        'Keyword Text': research_keyword_text,
        'Bid': bid,
        'Match Type': research_match_type}
    df_research = pd.concat([df_research, pd.DataFrame([new_research_keyword_row])], ignore_index=True)

## *Performance Campaign* ##

### Xây dựng bộ keyword chạy quảng cáo của Performance Campaign ###

In [ ]:
performance_keyword = pd.concat([lv1_keyword, lv2_keyword], ignore_index=True)
performance_keyword['Group'] = performance_keyword['Group'].str.lower()
performance_keyword['MatchType'] = performance_keyword['Group'].astype(str).apply(lambda x: 'exact' if 'spf' in x else '')

In [ ]:
performance_keyword_condition = performance_keyword['Keyword'].astype(str).apply(lambda x: len(x.split())) > 10
performance_keyword = performance_keyword[~performance_keyword_condition]

### Xây dựng data quảng cáo của Performance Campaign ###

In [ ]:
# Tạo DataFrame từ dict
df_performance = pd.DataFrame(data)

#Entity = "Campaign"
performance_campaign_name = product_name + " " + SKU + " - Performance"
performance_campaign_row = {
    'Entity': "Campaign",
    'Campaign Id': performance_campaign_name,
    'Portfolio Id': portfolio,
    'Campaign Name': performance_campaign_name,
    'Start Date': start_date_float,
    'Targeting Type': "Manual",
    'Daily Budget': 10,
    'Bidding Strategy': "Dynamic bids - down only"}

#Entity = "Ad group"
performance_ad_group_row = {
    'Entity': "Ad group",
    'Campaign Id': performance_campaign_name,
    'Ad Group Id': performance_campaign_name,
    'Ad Group Name': performance_campaign_name,
    'Ad Group Default Bid': 0.75}

#Entity = "Product ad"
performance_product_ad_row = {
    'Entity': "Product ad",
    'Campaign Id': performance_campaign_name,
    'Ad Group Id': performance_campaign_name,
    'sku': SKU}

df_performance = pd.concat([df_performance, pd.DataFrame([performance_campaign_row, performance_ad_group_row, performance_product_ad_row])], ignore_index=True)

#Entiry = 'Keyword'
for index, row in performance_keyword.iterrows():
    if row['MatchType'] == 'exact':
        performance_keyword_text = row['Keyword']
        performance_match_type = row['MatchType']
        new_performance_keyword_row = {
            'Entity': 'Keyword',
            'Campaign Id': performance_campaign_name,
            'Ad Group Id': performance_campaign_name,
            'Keyword Text': performance_keyword_text,
            'Bid': 0.5,
            'Match Type': performance_match_type}

        df_performance = pd.concat([df_performance, pd.DataFrame([new_performance_keyword_row])], ignore_index=True)

## *SP Campaign* ##

### Xây dựng bộ keyword chạy quảng cáo của SP Campaign ###

In [ ]:
def process_dataframes(df):
    while not df.empty:
        largest_group = df['Group'].iloc[-1]
        data = df[df['Group'] == largest_group].copy()
        df = df[df['Group'] != largest_group]

        if 'data2' in locals():
            data = pd.concat([data, data2], ignore_index=True)

        data['Ad Group'] = data['Group'].iloc[0]
        data['MatchType'] = 'phrase'
        data.loc[data['Group'] != data['Ad Group'], 'MatchType'] = 'negative exact'

        data2 = data[['Keyword', 'Group']].copy()
        yield data

# LV1 Keyword Processing
lv1_keyword_all_dataframes = list(process_dataframes(lv1_keyword))
lv1_final_dataframe = pd.concat(lv1_keyword_all_dataframes, ignore_index=True) if lv1_keyword_all_dataframes else None

# LV2 Keyword Processing
lv2_keyword_all_dataframes = list(process_dataframes(lv2_keyword))
lv2_final_dataframe = pd.concat(lv2_keyword_all_dataframes, ignore_index=True) if lv2_keyword_all_dataframes else None

if lv1_final_dataframe is not None:
    sp_keyword = pd.concat([lv1_final_dataframe, lv2_final_dataframe], ignore_index=True)
else:
    sp_keyword = lv2_final_dataframe
sp_keyword['bid'] = sp_keyword['MatchType'].apply(lambda x: 0.5 if 'phrase' in x else '')

In [ ]:
sp_keyword_condition = sp_keyword['Keyword'].astype(str).apply(lambda x: len(x.split())) > 10
sp_keyword = sp_keyword[~sp_keyword_condition]

### Xây dựng data quảng cáo của SP Campaign ###

In [ ]:
# Tạo DataFrame từ dict
df_sp = pd.DataFrame(data)

#Entity = "Campaign"
sp_campaign_name = product_name + " " + SKU + " SP"
sp_campaign_row = {
    'Entity': "Campaign",
    'Campaign Id': sp_campaign_name,
    'Portfolio Id': portfolio,
    'Campaign Name': sp_campaign_name,
    'Start Date': start_date_float,
    'Targeting Type': "Manual",
    'Daily Budget': 10,
    'Bidding Strategy': "Dynamic bids - down only"}

# Entity = "Bidding adjustment"
sp_bid_row = {
    'Entity': "Bidding adjustment",
    'Campaign Id': sp_campaign_name,
    'Placement' : 'placement top',
    'Percentage': 50}

# Tạo DataFrame mới từ các dòng sp_campaign_row và sp_bid_row
df_sp = pd.concat([df_sp, pd.DataFrame([sp_campaign_row, sp_bid_row])], ignore_index=True)

#Entity = "Ad group"
distinct_ad_groups = sp_keyword['Ad Group'].unique()

# Tạo DataFrame mới từ các dòng ad_group_row
ad_group_rows = []
for ad_group in distinct_ad_groups:
    ad_group_row = {
        'Entity': 'Ad group',
        'Campaign Id': sp_campaign_name,
        'Ad Group Id': ad_group,
        'Ad Group Name': ad_group,
        'Ad Group Default Bid': 0.75}
    ad_group_rows.append(ad_group_row)

df_sp = pd.concat([df_sp, pd.DataFrame(ad_group_rows)], ignore_index=True)

#Entity = "Product ad"
product_ad_rows = []
for ad_group in distinct_ad_groups:
    prorduct_ad_row = {
        'Entity': 'Product ad',
        'Campaign Id': sp_campaign_name,
        'Ad Group Id': ad_group,
        'sku': SKU}
    product_ad_rows.append(prorduct_ad_row)

df_sp = pd.concat([df_sp, pd.DataFrame(product_ad_rows)], ignore_index=True)

# Entity = "Negative Keyword"
for index, row in negative_keyword.iterrows():
    negative_keyword_text = row['Keyword']
    negative_match_type = row['MatchType']
    negative_row = {
        'Entity': 'Negative keyword',
        'Campaign Id': sp_campaign_name,
        'Keyword Text': negative_keyword_text,
        'Match Type': negative_match_type}
    df_sp = pd.concat([df_sp, pd.DataFrame([negative_row])], ignore_index=True)

# Entity = "Keyword"
for index, row in sp_keyword.iterrows():
    sp_keyword_text = row['Keyword']
    sp_match_type = row['MatchType']
    sp_ad_group = row['Ad Group']
    bid = row['bid']
    new_sp_keyword_row = {
        'Entity': 'Keyword',
        'Campaign Id': sp_campaign_name,
        'Ad Group Id': sp_ad_group,
        'Keyword Text': sp_keyword_text,
        'Bid': bid,
        'Match Type': sp_match_type}
    df_sp = pd.concat([df_sp, pd.DataFrame([new_sp_keyword_row])], ignore_index=True)

df_sp['Keyword Text'] = df_sp['Keyword Text'].astype(str)
negative_keyword_condition = (df_sp['Entity'] == 'Keyword') & (df_sp['Match Type'] == 'negative exact') & (df_sp['Keyword Text'].apply(lambda x: len(x.split()) > 4))
index_to_drop = df_sp.loc[negative_keyword_condition].index
df_sp.drop(index_to_drop, inplace=True)

# *SP PT Campaign* #

### Xây dựng data quảng cáo của SP PT Campaign ###

In [ ]:
df_sp_pt = pd.DataFrame(data)
sp_pt_campaign_name = product_name + " " + SKU + " PT"

if not competitor_asin.empty:
  #Entity = "Campaign"
  sp_pt_campaign_row = {
      'Entity': "Campaign",
      'Campaign Id': sp_pt_campaign_name,
      'Portfolio Id': portfolio,
      'Campaign Name': sp_pt_campaign_name,
      'Start Date': start_date_float,
      'Targeting Type': "Manual",
      'Daily Budget': 10,
      'Bidding Strategy': "Dynamic bids - down only"}

  # Entity = "Bidding adjustment"
  sp_pt_bid_row = {
    'Entity': "Bidding adjustment",
    'Campaign Id': sp_pt_campaign_name,
    'Placement' : 'placement product page',
    'Percentage': 50}

  #Entity = "Ad group"
  sp_pt_ad_group_row = {
      'Entity': "Ad group",
      'Campaign Id': sp_pt_campaign_name,
      'Ad Group Id': sp_pt_campaign_name,
      'Ad Group Name': sp_pt_campaign_name,
      'Ad Group Default Bid': 0.75}

  #Entity = "Product ad"
  sp_pt_product_ad_row = {
      'Entity': "Product ad",
      'Campaign Id': sp_pt_campaign_name,
      'Ad Group Id': sp_pt_campaign_name,
      'sku': SKU}

  df_sp_pt = pd.concat([df_sp_pt, pd.DataFrame([sp_pt_campaign_row, sp_pt_bid_row, sp_pt_ad_group_row, sp_pt_product_ad_row])], ignore_index=True)

  #Entiry = 'Product targeting'
  for index, row in competitor_asin.iterrows():
      sp_pt_asin_row = row['SP_ASIN']
      sp_pt_asin_rows = {
          'Entity': 'Product targeting',
          'Campaign Id': sp_pt_campaign_name,
          'Ad Group Id': sp_pt_campaign_name,
          'Bid': 0.5,
          'Product Targeting Expression': sp_pt_asin_row}
      df_sp_pt = pd.concat([df_sp_pt, pd.DataFrame([sp_pt_asin_rows])], ignore_index=True)

## *Gộp Dữ Liệu Các SP Campaign* ##

In [ ]:
sp_final_data = pd.concat([df_auto, df_research, df_performance, df_sp, df_sp_pt], ignore_index=True)
sp_final_data.loc[sp_final_data['Match Type'] == 'negative exact', 'Entity'] = 'Negative keyword'
sp_final_data.loc[sp_final_data['Match Type'] == 'negative phrase', 'Entity'] = 'Campaign negative keyword'
sp_final_data['Product'].fillna('Sponsored Products', inplace=True)
sp_final_data['Operation'].fillna('Create', inplace=True)
sp_final_data['State'].fillna('Enabled', inplace=True)
sp_final_data = sp_final_data.drop_duplicates()

# **SPONSORED BRANDS** #

## *Format Link Ảnh và Link Video* ##

In [ ]:
def creative_process(asset_type):
    start_index = asset_type.find("preview/") + len("preview/")
    end_index = asset_type.find("?entityId")
    desired_string = asset_type[start_index:end_index].replace("/version", ":version")
    return desired_string

custom_image = creative_process(image)
custom_video = creative_process(video)

## *SB Campaign* ##

### Xây dựng bộ keyword chạy quảng cáo của SB Campaign ###

In [ ]:
sb_phrase_keyword = pd.concat([lv1_keyword, lv2_keyword], ignore_index=True)
sb_phrase_keyword['MatchType'] = 'phrase'

In [ ]:
sb_phrase_keyword_condition = sb_phrase_keyword['Keyword'].astype(str).apply(lambda x: len(x.split())) > 10
sb_phrase_keyword = sb_phrase_keyword[~sb_phrase_keyword_condition]

### Xây dựng data quảng cáo của SB Campaign ###

In [ ]:
df_camp_sb = pd.DataFrame(data_sb)
sb_campaign_name = product_name + " " + SKU + " SB"

# Entity = "Campaign"
sb_campaign_row = {
    'Entity': "Campaign",
    'Campaign ID': sb_campaign_name,
    'Portfolio ID': portfolio,
    'Campaign name': sb_campaign_name,
    'Start date': start_date_float,
    'Brand Entity ID': brand_entity,
    'Budget type': 'Daily',
    'Budget': 10,
    'Bid optimization': "TRUE"}

# Entity = "Ad group"
sb_ad_group_row = {
    'Entity': "Ad group",
    'Campaign ID': sb_campaign_name,
    'Ad Group ID': sb_campaign_name,
    'Ad group name': sb_campaign_name}

# Entity = "Product collection ad"
sb_collection_ad_row = {
    'Entity': "Product collection ad",
    'Campaign ID': sb_campaign_name,
    'Ad Group ID': sb_campaign_name,
    'Ad name': sb_campaign_name,
    'Landing page ASINs': target_asins,
    'Landing page type': 'Product list',
    'Brand name': 'Canamax',
    'Brand logo asset ID': brand_logo,
    'Custom images': '[{{"assetId":"{}"}}]'.format(custom_image),
    'Creative headline': 'On Time, Every Time',
    'Creative ASINs': target_asins[:34]}

df_camp_sb = pd.concat([df_camp_sb, pd.DataFrame([sb_campaign_row, sb_ad_group_row, sb_collection_ad_row])], ignore_index=True)

# Entity = "Negative keyword"
for index, row in negative_keyword.iterrows():
    negative_keyword_text = row['Keyword']
    negative_match_type = row['MatchType']
    negative_row = {
        'Entity': 'Negative keyword',
        'Campaign ID': sb_campaign_name,
        'Ad Group ID': sb_campaign_name,
        'Keyword text': negative_keyword_text,
        'Match type': negative_match_type}
    df_camp_sb = pd.concat([df_camp_sb, pd.DataFrame([negative_row])], ignore_index=True)

# Entity = "Keyword"
for index, row in sb_phrase_keyword.iterrows():
    sb_keyword_text = row['Keyword']
    sb_match_type = row['MatchType']
    sb_keyword_row = {
        'Entity': 'Keyword',
        'Campaign ID': sb_campaign_name,
        'Ad Group ID': sb_campaign_name,
        'Keyword text': sb_keyword_text,
        'Bid': 0.5,
        'Match type': sb_match_type}
    df_camp_sb = pd.concat([df_camp_sb, pd.DataFrame([sb_keyword_row])], ignore_index=True)

## *Video Phrase Campaign* ##

### Xây dựng data quảng cáo của Video Phrase Campaign ###

In [ ]:
df_video_phrase = pd.DataFrame(data_sb)
video_phrase_name = product_name + " " + SKU + " Video Ads Phrase"

# Entity = "Campaign"
video_phrase_campaign_row = {
    'Entity': "Campaign",
    'Campaign ID': video_phrase_name,
    'Portfolio ID': portfolio,
    'Campaign name': video_phrase_name,
    'Start date': start_date_float,
    'Brand Entity ID': brand_entity,
    'Budget type': 'Daily',
    'Budget': 10,
    'Bid optimization': "TRUE"}

# Entity = "Ad group"
video_phrase_ad_group_row = {
    'Entity': "Ad group",
    'Campaign ID': video_phrase_name,
    'Ad Group ID': video_phrase_name,
    'Ad group name': video_phrase_name}

# Entity = "Video ad"
video_phrase_ad_row = {
    'Entity': "Video ad",
    'Campaign ID': video_phrase_name,
    'Ad Group ID': video_phrase_name,
    'Ad name': video_phrase_name,
    'Landing page URL': 'https://www.amazon.com/dp/' + asin,
    'Landing page type': 'Detail Page',
    'Brand name': 'Canamax',
    'Brand logo asset ID': brand_logo,
    'Consent to translate': 'False',
    'Video asset IDs': custom_video,
    'Creative ASINs': asin}

df_video_phrase = pd.concat([df_video_phrase, pd.DataFrame([video_phrase_campaign_row,
                                                            video_phrase_ad_group_row,
                                                            video_phrase_ad_row])], ignore_index=True)

# Entity = "Negative Keyword"
for index, row in negative_keyword.iterrows():
    negative_keyword_text = row['Keyword']
    negative_match_type = row['MatchType']
    negative_row = {
        'Entity': 'Negative keyword',
        'Campaign ID': video_phrase_name,
        'Ad Group ID': video_phrase_name,
        'Keyword text': negative_keyword_text,
        'Match type': negative_match_type}
    df_video_phrase = pd.concat([df_video_phrase, pd.DataFrame([negative_row])], ignore_index=True)

# Entity = "Keyword"
for index, row in sb_phrase_keyword.iterrows():
    sb_phrase_keyword_text = row['Keyword']
    sb_phrase_match_type = row['MatchType']
    sb_phrase_keyword_row = {
        'Entity': 'Keyword',
        'Campaign ID': video_phrase_name,
        'Ad Group ID': video_phrase_name,
        'Keyword text': sb_phrase_keyword_text,
        'Bid': 0.25,
        'Match type': sb_phrase_match_type}
    df_video_phrase = pd.concat([df_video_phrase, pd.DataFrame([sb_phrase_keyword_row])], ignore_index=True)

## *Video Broad Campaign* ##

### Xây dựng bộ keyword chạy quảng cáo của Video Broad Campaign ###

In [ ]:
sb_broad_keyword = pd.concat([lv1_keyword, lv2_keyword], ignore_index=True)
sb_broad_keyword['MatchType'] = 'broad'

In [ ]:
sb_broad_keyword_condition = sb_broad_keyword['Keyword'].astype(str).apply(lambda x: len(x.split())) > 10
sb_broad_keyword = sb_broad_keyword[~sb_broad_keyword_condition]

### Xây dựng data quảng cáo của Video Broad Campaign ###

In [ ]:
df_video_broad = pd.DataFrame(data_sb)
video_broad_name = product_name + " " + SKU + " Video Ads Broad"

# Entity = "Campaign"
video_broad_campaign_row = {
    'Entity': "Campaign",
    'Campaign ID': video_broad_name,
    'Portfolio ID': portfolio,
    'Campaign name': video_broad_name,
    'Start date': start_date_float,
    'Brand Entity ID': brand_entity,
    'Budget type': 'Daily',
    'Budget': 10,
    'Bid optimization': "TRUE"}

# Entity = "Ad group"
video_broad_ad_group_row = {
    'Entity': "Ad group",
    'Campaign ID': video_broad_name,
    'Ad Group ID': video_broad_name,
    'Ad group name': video_broad_name}

# Entity = "Video ad"
video_broad_ad_row = {
    'Entity': "Video ad",
    'Campaign ID': video_broad_name,
    'Ad Group ID': video_broad_name,
    'Ad name': video_broad_name,
    'Landing page URL': 'https://www.amazon.com/dp/' + asin,
    'Landing page type': 'Detail Page',
    'Brand name': 'Canamax',
    'Brand logo asset ID': brand_logo,
    'Consent to translate': 'False',
    'Video asset IDs': custom_video,
    'Creative ASINs': asin}

df_video_broad = pd.concat([df_video_broad, pd.DataFrame([video_broad_campaign_row,
                                                            video_broad_ad_group_row,
                                                            video_broad_ad_row])], ignore_index=True)

# Entity = "Negative Keyword"
for index, row in negative_keyword.iterrows():
    negative_keyword_text = row['Keyword']
    negative_match_type = row['MatchType']
    negative_row = {
        'Entity': 'Negative keyword',
        'Campaign ID': video_broad_name,
        'Ad Group ID': video_broad_name,
        'Keyword text': negative_keyword_text,
        'Match type': negative_match_type}
    df_video_broad = pd.concat([df_video_broad, pd.DataFrame([negative_row])], ignore_index=True)

sb_broad_video_negative = sb_phrase_keyword[sb_phrase_keyword['Keyword'].apply(lambda x: len(x.split()) <= 4)]

for index, row in sb_broad_video_negative.iterrows():
        sb_negative_exact = row['Keyword']
        sb_negative_exact_row = {
            'Entity': 'Negative keyword',
            'Campaign ID': video_broad_name,
            'Ad Group ID': video_broad_name,
            'Keyword text': sb_negative_exact,
            'Match type': 'Negative phrase'}
        df_video_broad = pd.concat([df_video_broad, pd.DataFrame([sb_negative_exact_row])], ignore_index=True)

# Entity = "Keyword"
for index, row in sb_broad_keyword.iterrows():
    sb_broad_keyword_text = row['Keyword']
    sb_broad_match_type = row['MatchType']
    sb_broad_keyword_row = {
        'Entity': 'Keyword',
        'Campaign ID': video_broad_name,
        'Ad Group ID': video_broad_name,
        'Keyword text': sb_broad_keyword_text,
        'Bid': 0.25,
        'Match type': sb_broad_match_type}
    df_video_broad = pd.concat([df_video_broad, pd.DataFrame([sb_broad_keyword_row])], ignore_index=True)

## *Video PT Campaign* ##

In [ ]:
df_video_pt = pd.DataFrame(data_sb)
video_pt_name = product_name + " " + SKU + " Video Ads PT"

# Entity = "Campaign"
video_pt_campaign_row = {
    'Entity': "Campaign",
    'Campaign ID': video_pt_name,
    'Portfolio ID': portfolio,
    'Campaign name': video_pt_name,
    'Start date': start_date_float,
    'Brand Entity ID': brand_entity,
    'Budget type': 'Daily',
    'Budget': 10,
    'Bid optimization': "TRUE"}

# Entity = "Ad group"
video_pt_ad_group_row = {
    'Entity': "Ad group",
    'Campaign ID': video_pt_name,
    'Ad Group ID': video_pt_name,
    'Ad group name': video_pt_name}

# Entity = "Video ad"
video_pt_ad_row = {
    'Entity': "Video ad",
    'Campaign ID': video_pt_name,
    'Ad Group ID': video_pt_name,
    'Ad name': video_pt_name,
    'Landing page URL': 'https://www.amazon.com/dp/' + asin,
    'Landing page type': 'Detail Page',
    'Brand name': 'Canamax',
    'Brand logo asset ID': brand_logo,
    'Consent to translate': 'False',
    'Video asset IDs': custom_video,
    'Creative ASINs': asin}

df_video_pt = pd.concat([df_video_pt, pd.DataFrame([video_pt_campaign_row, video_pt_ad_group_row, video_pt_ad_row])], ignore_index=True)

#Entiry = 'Product targeting'
for index, row in competitor_asin.iterrows():
  video_pt_asin_row = row['SD_ASIN']
  video_pt_rows = {
      'Entity': 'Product targeting',
      'Campaign ID': video_pt_name,
      'Ad Group ID': video_pt_name,
      'Bid': 0.3,
      'Product targeting expression': video_pt_asin_row}
  df_video_pt = pd.concat([df_video_pt, pd.DataFrame([video_pt_rows])], ignore_index=True)

## *Gộp Dữ Liệu Các SB Campaign* ##

In [ ]:
sb_final_data = pd.concat([df_camp_sb, df_video_phrase, df_video_broad, df_video_pt], ignore_index=True)
sb_final_data['Product'].fillna('Sponsored Brands', inplace=True)
sb_final_data['Operation'].fillna('Create', inplace=True)
sb_final_data['State'].fillna('Enabled', inplace=True)
sb_final_data = sb_final_data.drop_duplicates()

# **SPONSORED DISPLAY** #

## *SD PT Campaign* ##

### Xây dựng data quảng cáo của SD PT Campaign ###

In [ ]:
df_sd = pd.DataFrame(data)
sd_campaign_name = product_name + " " + SKU + " SD PT"

if not competitor_asin.empty:
  #Entity = "Campaign"
  sd_campaign_row = {
      'Entity': "Campaign",
      'Campaign Id': sd_campaign_name,
      'Portfolio Id': portfolio,
      'Campaign Name': sd_campaign_name,
      'Start Date': start_date_float,
      'Tactic': 'T00020',
      'Budget Type': 'Daily',
      'Budget': 10,
      'Cost Type': 'CPC'}

  # Entity = "Ad group"
  sd_ad_group_row = {
      'Entity': "Ad group",
      'Campaign Id': sd_campaign_name,
      'Ad Group Id': sd_campaign_name,
      'Ad Group Name': sd_campaign_name,
      'Tactic': 'T00020',
      'Ad Group Default Bid': 0.5,
      'Bid Optimization': 'Optimize for page visits'}

  #Entity = "Product ad"
  sd_product_ad_row = {
      'Entity': "Product ad",
      'Campaign Id': sd_campaign_name,
      'Ad Group Id': sd_campaign_name,
      'sku': SKU}

  df_sd = pd.concat([df_sd, pd.DataFrame([sd_campaign_row, sd_ad_group_row, sd_product_ad_row])], ignore_index=True)

  #Entiry = 'Product targeting'
  for index, row in competitor_asin.iterrows():
      sd_asin_row = row['SD_ASIN']
      sd_asin_rows = {
          'Entity': 'Contextual targeting',
          'Campaign Id': sd_campaign_name,
          'Ad Group Id': sd_campaign_name,
          'Bid': 0.5,
          'Targeting Expression': sd_asin_row}
      df_sd = pd.concat([df_sd, pd.DataFrame([sd_asin_rows])], ignore_index=True)

  df_sd['Product'].fillna('Sponsored Display', inplace=True)
  df_sd['Operation'].fillna('Create', inplace=True)
  df_sd['State'].fillna('Enabled', inplace=True)
  df_sd = df_sd.drop_duplicates()

# **TẢI DỮ LIỆU** #

In [ ]:
output_product_name = process_text(product_name)
with pd.ExcelWriter(output_product_name + "_campaigns.xlsx") as writer:
    sp_final_data.to_excel(writer, sheet_name='Sponsored Products Campaigns', index=False)
    sb_final_data.to_excel(writer, sheet_name='SB Multi Ad Group Campaigns', index=False)
    df_sd.to_excel(writer, sheet_name="Sponsored Display Campaigns", index=False)

files.download(output_product_name + "_campaigns.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>